https://towardsdatascience.com/from-ml-model-to-ml-pipeline-9f95c32c6512

https://github.com/Mykrass/ml_pipeline

In [ ]:
!git clone https://github.com/Mykrass/ml_pipeline

Cloning into 'ml_pipeline'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 54 (delta 27), reused 33 (delta 11), pack-reused 0
Receiving objects: 100% (54/54), 10.25 KiB | 3.42 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [ ]:
!ls -la ./ml_pipeline/

total 40
drwxr-xr-x 3 root root 4096 Nov  3 11:15 .
drwxr-xr-x 1 root root 4096 Nov  3 11:15 ..
-rw-r--r-- 1 root root  322 Nov  3 11:15 functions.py
drwxr-xr-x 8 root root 4096 Nov  3 11:15 .git
-rw-r--r-- 1 root root   39 Nov  3 11:15 .gitignore
-rw-r--r-- 1 root root 8856 Nov  3 11:15 ml_pipeline.ipynb
-rw-r--r-- 1 root root 1413 Nov  3 11:15 README.md
-rw-r--r-- 1 root root 2888 Nov  3 11:15 transformers.py


In [ ]:
# Customer functions
!python3 ./ml_pipeline/functions.py --help
!cat ./ml_pipeline/functions.py

from sklearn.metrics import roc_auc_score

def calculate_roc_auc(model_pipe, X, y):
    """Calculate roc auc score. 
    
    Parameters:
    model_pipe: sklearn model or pipeline
    X: features
    y: true target
    """
    y_proba = model_pipe.predict_proba(X)[:,1]
    return roc_auc_score(y, y_proba)

In [ ]:
# Customer functions
!python3 ./ml_pipeline/transformers.py --help
!cat ./ml_pipeline/transformers.py

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder

class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_transformed = X.copy()
        #X_transformed['adult_male'] = (X['sex']=='male') & ~(X['age']<16)
        X_transformed['child'] = (X['MARRIAGE']=='1') & ~(X['AGE']<16)
        #X_transformed['who'] = np.where(X['age']<16, 'child', 
        #                                np.where(X['sex']=='female', 'woman', 'man'))
        return X

class Imputer(BaseEstimator, TransformerMixin):
    def __init__(self, features, method='constant', value='missing'):
        self.features = features
        self.method = method
        self.value = value
    
    def fit(self, X, y=None):
        if self.method=='mean':
            self.value = X[self.features].mean()
        re

# 🔧 Load libraries and data
We will assume features that are included in `columns` are not available in the raw data. This gives us an opportunity to build FeatureExtractor custom transformer to extract useful features in the ML Pipeline.

In [ ]:
# PyCaret
!git clone https://github.com/Mykrass/pycaret.git
!ls -GFlash --color ./pycaret/datasets

Cloning into 'pycaret'...
remote: Enumerating objects: 4020, done.
remote: Total 4020 (delta 0), reused 0 (delta 0), pack-reused 4020
Receiving objects: 100% (4020/4020), 57.68 MiB | 13.29 MiB/s, done.
Resolving deltas: 100% (2659/2659), done.
Updating files: 100% (278/278), done.
total 49M
 4.0K drwxr-xr-x  2 root  4.0K Nov  3 11:15 ./
 4.0K drwxr-xr-x 12 root  4.0K Nov  3 11:15 ../
 3.5M -rw-r--r--  1 root  3.5M Nov  3 11:15 amazon.csv
 120K -rw-r--r--  1 root  117K Nov  3 11:15 anomaly.csv
 8.0K -rw-r--r--  1 root  7.6K Nov  3 11:15 asia_gdp.csv
  28K -rw-r--r--  1 root   25K Nov  3 11:15 automobile.csv
 3.6M -rw-r--r--  1 root  3.6M Nov  3 11:15 bank.csv
1008K -rw-r--r--  1 root 1008K Nov  3 11:15 bike.csv
  12K -rw-r--r--  1 root   12K Nov  3 11:15 blood.csv
  36K -rw-r--r--  1 root   35K Nov  3 11:15 boston.csv
  16K -rw-r--r--  1 root   15K Nov  3 11:15 cancer.csv
  60K -rw-r--r--  1 root   58K Nov  3 11:15 concrete.csv
  12K -rw-r--r--  1 root  9.1K Nov  3 11:15 country-data.cs

In [ ]:
!head ./pycaret/datasets/credit.csv

LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
50000,1,1,2,37,0,0,0,0,0,0,64400,57069,57608,19394,19619,20024,2500,1815,657,1000,1000,800,0
100000,2,2,2,23,0,-1,-1,0,0,-1,11876,380,601,221,-159,567,380,601,0,581,1687,1542,0
140000,2,3,1,28,0,0,2,0,0,0,11285,14096,12108,12211,11793,3719,3329,0,432,1000,1000,1000,0
20000,1,3,2,35,-2,-2,-2,-2,-1,-1,0,0,0,0,13007,13912,0,0,0,13007,1122,0,0
200000,2,3,2,34,0,0,2,0,0,-1,11073,9787,5535,2513,1828,3731,2306,12,50,300,3738,66,0


In [ ]:
import pandas as pd
df_raw= pd.read_csv('./pycaret/datasets/credit.csv')
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 24 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LIMIT_BAL  24000 non-null  int64  
 1   SEX        24000 non-null  int64  
 2   EDUCATION  24000 non-null  int64  
 3   MARRIAGE   24000 non-null  int64  
 4   AGE        24000 non-null  int64  
 5   PAY_1      24000 non-null  int64  
 6   PAY_2      24000 non-null  int64  
 7   PAY_3      24000 non-null  int64  
 8   PAY_4      24000 non-null  int64  
 9   PAY_5      24000 non-null  int64  
 10  PAY_6      24000 non-null  int64  
 11  BILL_AMT1  24000 non-null  float64
 12  BILL_AMT2  24000 non-null  float64
 13  BILL_AMT3  24000 non-null  float64
 14  BILL_AMT4  24000 non-null  float64
 15  BILL_AMT5  24000 non-null  float64
 16  BILL_AMT6  24000 non-null  float64
 17  PAY_AMT1   24000 non-null  float64
 18  PAY_AMT2   24000 non-null  float64
 19  PAY_AMT3   24000 non-null  float64
 20  PAY_AM

In [ ]:
df_raw.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')

In [ ]:
# Way to function
import sys
sys.path.append('/content/ml_pipeline/')

# Data manipulation
from seaborn import load_dataset
import numpy as np
import pandas as pd
from functions import calculate_roc_auc
pd.options.display.precision = 4
pd.options.mode.chained_assignment = None

# Machine learning pipeline
from sklearn.model_selection import train_test_split
from transformers import FeatureExtractor, Imputer, CardinalityReducer, Encoder
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Load data
select = ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default']
df = df_raw[select].copy()
df.iloc[:, 1:4] = df.iloc[:, 1:4].astype('object')
print(df.shape)
df.head()

(24000, 24)


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,90000,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
2,50000,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
3,50000,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
4,50000,1,1,2,37,0,0,0,0,0,...,19394.0,19619.0,20024.0,2500.0,1815.0,657.0,1000.0,1000.0,800.0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 24 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LIMIT_BAL  24000 non-null  int64  
 1   SEX        24000 non-null  object 
 2   EDUCATION  24000 non-null  object 
 3   MARRIAGE   24000 non-null  object 
 4   AGE        24000 non-null  int64  
 5   PAY_1      24000 non-null  int64  
 6   PAY_2      24000 non-null  int64  
 7   PAY_3      24000 non-null  int64  
 8   PAY_4      24000 non-null  int64  
 9   PAY_5      24000 non-null  int64  
 10  PAY_6      24000 non-null  int64  
 11  BILL_AMT1  24000 non-null  float64
 12  BILL_AMT2  24000 non-null  float64
 13  BILL_AMT3  24000 non-null  float64
 14  BILL_AMT4  24000 non-null  float64
 15  BILL_AMT5  24000 non-null  float64
 16  BILL_AMT6  24000 non-null  float64
 17  PAY_AMT1   24000 non-null  float64
 18  PAY_AMT2   24000 non-null  float64
 19  PAY_AMT3   24000 non-null  float64
 20  PAY_AM

Rather than defining feature groups manually, we will divide variables into two categories depending on their type.

In [ ]:
SEED = 42
TARGET = 'default'
FEATURES = df.columns.drop(TARGET)

NUMERICAL = df[FEATURES].select_dtypes('number').columns
print(f"Numerical features: {', '.join(NUMERICAL)}")

CATEGORICAL = pd.Index(np.setdiff1d(FEATURES, NUMERICAL))
print(f"Categorical features: {', '.join(CATEGORICAL)}\n")

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=TARGET), df[TARGET],
                                                    test_size=.2, random_state=SEED,
                                                    stratify=df[TARGET])

print(f"Training features shape: {X_train.shape}")
print(f"Test features shape: {X_test.shape}")

Numerical features: LIMIT_BAL, AGE, PAY_1, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6
Categorical features: EDUCATION, MARRIAGE, SEX

Training features shape: (19200, 23)
Test features shape: (4800, 23)


In [ ]:
# NUMERICAL
df[FEATURES].select_dtypes('number')[:5]

,LIMIT_BAL,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,20000,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0
1,90000,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0
2,50000,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0
3,50000,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0
4,50000,37,0,0,0,0,0,0,64400.0,57069.0,57608.0,19394.0,19619.0,20024.0,2500.0,1815.0,657.0,1000.0,1000.0,800.0


In [ ]:
# OBJECT
df[FEATURES].select_dtypes('O')[:5]

,SEX,EDUCATION,MARRIAGE
0,2,2,1
1,2,2,2
2,2,2,1
3,1,2,1
4,1,1,2


In [ ]:
X_train[:5]

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
21909,50000,1,3,1,44,0,0,0,0,0,...,15816.0,13983.0,13638.0,10657.0,1282.0,1253.0,1170.0,535.0,387.0,405.0
3055,80000,1,3,2,43,7,6,5,4,3,...,76767.0,75393.0,74731.0,23207.0,0.0,0.0,0.0,0.0,0.0,2000.0
19770,170000,1,2,2,36,-2,-2,-2,-2,-2,...,0.0,1340.0,0.0,0.0,1598.0,0.0,1340.0,0.0,0.0,3200.0
12119,160000,2,2,2,23,0,0,0,0,-1,...,17164.0,7175.0,1942.0,100025.0,2000.0,3181.0,3012.0,1942.0,100025.0,3000.0
22644,210000,2,2,1,31,0,0,0,0,0,...,203831.0,178410.0,130619.0,115700.0,7736.0,7100.0,8300.0,4800.0,4396.0,4200.0


In [ ]:
y_train[:5]

21909    0
3055     1
19770    0
12119    0
22644    0
Name: default, dtype: int64

# 💻 Build ML Pipeline
This pipeline is here to illustrate how custom features can be useful and not currently optimised for speed.

In [ ]:
pipe = Pipeline([
    ('feature_extractor', FeatureExtractor()),
    ('cat_imputer', Imputer(CATEGORICAL)),
    ('cardinality_reducer', CardinalityReducer(CATEGORICAL, threshold=0.1)),
    ('encoder', Encoder(CATEGORICAL)),
    ('num_imputer', Imputer(NUMERICAL, method='mean')),
    ('feature_selector', RFE(LogisticRegression(random_state=SEED, max_iter=500), n_features_to_select=8)),
    ('model', LogisticRegression(random_state=SEED, max_iter=500))
])

pipe.fit(X_train, y_train)
print(f"Train ROC-AUC: {calculate_roc_auc(pipe, X_train, y_train):.4f}")
print(f"Test ROC-AUC: {calculate_roc_auc(pipe, X_test, y_test):.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Train ROC-AUC: 0.7098
Test ROC-AUC: 0.6999


In [ ]:
pipe.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('feature_extractor', FeatureExtractor()),
                ('cat_imputer',
                 Imputer(features=Index(['EDUCATION', 'MARRIAGE', 'SEX'], dtype='object'))),
                ('cardinality_reducer',
                 CardinalityReducer(features=Index(['EDUCATION', 'MARRIAGE', 'SEX'], dtype='object'))),
                ('encoder',
                 Encoder(features=Index(['EDUCATION', 'MARRIAGE', 'SEX'], dtype='object'))),
                ('num_imputer',
                 Im...
BILL_AMT3     46828.1155
BILL_AMT4     43217.6259
BILL_AMT5     40302.6929
BILL_AMT6     38845.2677
PAY_AMT1       5595.0001
PAY_AMT2       5809.5891
PAY_AMT3       5255.3124
PAY_AMT4       4812.4696
PAY_AMT5       4792.9787
PAY_AMT6       5341.4207
dtype: float64)),
                ('feature_selector',
                 RFE(estimator=LogisticRegression(max_iter=500,
                                                  random_state=42),
                     n_features_to_select=8)),
                ('model', LogisticRegression(max_iter=500, random_state=42))])

# 🔎 Explore pipeline
We will check top features as an example.

In [ ]:
top_features = pipe['feature_selector'].feature_names_in_[pipe['feature_selector'].support_]
print(f"Top {len(top_features)} features: {', '.join(top_features)}")

Top 8 features: PAY_1, PAY_2, PAY_3, PAY_5, EDUCATION_other, MARRIAGE_2, MARRIAGE_other, SEX_2


# 🔎 Explore plot
We will check top features with plot.

In [ ]:
# Standard
import pandas as pd
import numpy as np
import os
# Plots
from plotly.offline import iplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
# Sklearn tools
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import *
# Extras
from datetime import date
import warnings
warnings.filterwarnings("ignore")

# Helper functions for structured data
## Get info about the dataset
def dataset_info(dataset, dataset_name: str):
    print(f"Dataset Name: {dataset_name} \
        | Number of Samples: {dataset.shape[0]} \
        | Number of Columns: {dataset.shape[1]}")
    print(30*"=")
    print("Column             Data Type")
    print(dataset.dtypes)
    print(30*"=")
    missing_data = dataset.isnull().sum()
    if sum(missing_data) > 0:
        print(missing_data[missing_data.values > 0])
    else:
        print("No Missing Data on this Dataset!")
    print(30*"=")
    print("Memory Usage: {} MB".\
         format(np.round(
         dataset.memory_usage(index=True).sum() / 10e5, 3
         )))
## Dataset Sampling
def data_sampling(dataset, frac: float, random_seed: int):
    data_sampled_a = dataset.sample(frac=frac, random_state=random_seed)
    data_sampled_b =  dataset.drop(data_sampled_a.index).reset_index(drop=True)
    data_sampled_a.reset_index(drop=True, inplace=True)
    return data_sampled_a, data_sampled_b
## Bar Plot
def bar_plot(data, plot_title: str, x_axis: str, y_axis: str):
    colors = ["#0080ff",] * len(data)
    colors[0] = "#ff8000"
    trace = go.Bar(y=data.values, x=data.index, text=data.values,
                    marker_color=colors)
    layout = go.Layout(autosize=False, height=600,
                    title={"text" : plot_title,
                       "y" : 0.9,
                       "x" : 0.5,
                       "xanchor" : "center",
                       "yanchor" : "top"},
                    xaxis={"title" : x_axis},
                    yaxis={"title" : y_axis},)
    fig = go.Figure(data=trace, layout=layout)
    fig.update_layout(template="simple_white")
    fig.update_traces(textposition="outside",
                    textfont_size=14,
                    marker=dict(line=dict(color="#000000", width=2)))
    fig.update_yaxes(automargin=True)
    iplot(fig)
## Plot Pie Chart
def pie_plot(data, plot_title: str):
    trace = go.Pie(labels=data.index, values=data.values)
    layout = go.Layout(autosize=False,
                    title={"text" : plot_title,
                       "y" : 0.9,
                       "x" : 0.5,
                       "xanchor" : "center",
                       "yanchor" : "top"})
    fig = go.Figure(data=trace, layout=layout)
    fig.update_traces(textfont_size=14,
                    marker=dict(line=dict(color="#000000", width=2)))
    fig.update_yaxes(automargin=True)
    iplot(fig)
## Histogram
def histogram_plot(data, plot_title: str, y_axis: str):
    trace = go.Histogram(x=data)
    layout = go.Layout(autosize=False,
                    title={"text" : plot_title,
                       "y" : 0.9,
                       "x" : 0.5,
                       "xanchor" : "center",
                       "yanchor" : "top"},
                    yaxis={"title" : y_axis})
    fig = go.Figure(data=trace, layout=layout)
    fig.update_traces(marker=dict(line=dict(color="#000000", width=2)))
    fig.update_layout(template="simple_white")
    fig.update_yaxes(automargin=True)
    iplot(fig)
# Particular case: Histogram subplot (1, 2)
def histogram_subplot(dataset_a, dataset_b, feature_a: str,
                        feature_b: str, title: str, title_a: str, title_b: str):
    fig = make_subplots(rows=1, cols=2, subplot_titles=(
                        title_a,
                        title_b
                        )
                    )
    fig.add_trace(go.Histogram(x=dataset_a[feature_a],
                               showlegend=False),
                                row=1, col=1)
    fig.add_trace(go.Histogram(x=dataset_b[feature_b],
                               showlegend=False),
                              row=1, col=2)
    fig.update_layout(template="simple_white")
    fig.update_layout(autosize=False,
                        title={"text" : title,
                        "y" : 0.9,
                        "x" : 0.5,
                        "xanchor" : "center",
                        "yanchor" : "top"},
                        yaxis={"title" : "<i>Frequency</i>"})
    fig.update_traces(marker=dict(line=dict(color="#000000", width=2)))
    fig.update_yaxes(automargin=True)
    iplot(fig)
# Calculate scores with Test/Unseen labeled data
def test_score_report(data_unseen, predict_unseen):
    le = LabelEncoder()
    data_unseen["Label"] = le.fit_transform(data_unseen.Churn.values)
    data_unseen["Label"] = data_unseen["Label"].astype(int)
    accuracy = accuracy_score(data_unseen["Label"], predict_unseen["Label"])
    roc_auc = roc_auc_score(data_unseen["Label"], predict_unseen["Label"])
    precision = precision_score(data_unseen["Label"], predict_unseen["Label"])
    recall = recall_score(data_unseen["Label"], predict_unseen["Label"])
    f1 = f1_score(data_unseen["Label"], predict_unseen["Label"])

In [ ]:
df.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')

In [ ]:
pie_plot(df["default"].value_counts(),
         plot_title="<b>Client Churn Distribution<b>")

In [ ]:
pie_plot(df["SEX"].value_counts(),
         plot_title="<b>Client Gender Distribution<b>")

In [ ]:
pie_plot(df["MARRIAGE"].value_counts(),
         plot_title="<b>Client Marriage Distribution<b>")

In [ ]:
df_aux = df_raw.query('(AGE >0) and (default == 0)')
df_aux = df_aux["AGE"].value_counts()
bar_plot(df_aux, "<b>Age Churned Clients</b>",
         "<i>Age</i>", "<i>Count</i>")

In [ ]:
df_aux = df_raw.query('(AGE >0) and (default == 1)')
df_aux = df_aux["AGE"].value_counts()
bar_plot(df_aux, "<b>Age No Churned Clients</b>",
         "<i>Age</i>", "<i>Count</i>")

In [ ]:
df_aux = df_raw.query('(LIMIT_BAL >0) and (default == 0)')
df_aux = df_aux["LIMIT_BAL"].value_counts()
bar_plot(df_aux, "<b>LIMIT_BAL Churned Clients</b>",
         "<i>LIMIT_BAL</i>", "<i>Count</i>")

In [ ]:
df_aux = df_raw.query('(LIMIT_BAL >0) and (default == 1)')
df_aux = df_aux["LIMIT_BAL"].value_counts()
bar_plot(df_aux, "<b>LIMIT_BAL No Churned Clients</b>",
         "<i>LIMIT_BAL</i>", "<i>Count</i>")

In [ ]:
df_aux = df_raw.query('(LIMIT_BAL >0) and (default == 0)')
histogram_subplot(df_aux, df_aux, "LIMIT_BAL", "EDUCATION",
                  "<b>Distribution for Churned Clients</b>",
                  "(a) Limit Distribution", "(b) Education Distribution")

In [ ]:
df_aux = df_raw.query('(LIMIT_BAL >0) and (default == 0)')
df_aux = df_aux["EDUCATION"].value_counts()
bar_plot(df_aux, "<b>Education Churned Clients</b>",
         "<i>Education</i>", "<i>Count</i>")

In [ ]:
df_aux = df_raw.query('(LIMIT_BAL >0) and (default == 1)')
histogram_subplot(df_aux, df_aux, "LIMIT_BAL", "EDUCATION",
                  "<b>Distribution for No Churned Clients</b>",
                  "(a) Limit Distribution", "(b) Education Distribution")

In [ ]:
df_aux = df_raw.query('(LIMIT_BAL >0) and (default == 1)')
df_aux = df_aux["EDUCATION"].value_counts()
bar_plot(df_aux, "<b>Education No Churned Clients</b>",
         "<i>Education</i>", "<i>Count</i>")

THE END